In [1]:
import numpy as np
np.set_printoptions(threshold=np.nan, linewidth=np.nan)

In [2]:
import pickle
with open('df.pickle', mode='rb') as h:
    _, df_test = pickle.load(h)
with open('hex.pickle', mode='rb') as h:
    hex_data = pickle.load(h)
labels = df_test['label']
name_id = hex_data['name_id']
id_hls = map(set, hex_data['id_hierarchical_labels'])  # id_hls for id_hierarchical_labels
id_fh = map(lambda x: max(x, key=len), id_hls)  # id_fh for id_full_hierarchy
state_space = hex_data['state_space']

In [3]:
def get_accuracy(Y):
    """
    Type 1: Is the full hierarchy
    Type 2: Covers the full hierarchy
    Type 3: Is any partial hierarchy
    Type 4: Covers any partial hierarchy
    Implication: 1->2, 1->3, 2->4, 3->4
    """
    accuracy = np.zeros(5, dtype=np.float32)
    for i in range(0, len(Y)):
        y = Y[i]
        if np.argmax(y[:20]) == labels[i]:
            accuracy[0] += 1
        y = np.nonzero((y / y.max()) > threshold)[0]  # designed for probs, also fine for binary
        if tuple(y) == id_fh[labels[i]]:
            accuracy[1:] += 1
        elif set(id_fh[labels[i]]) in set(y):
            accuracy[[2, 4]] += 1
        elif tuple(y) in id_hls[labels[i]]:
            accuracy[[3, 4]] += 1
        elif any(set(x) in set(y) for x in id_hls[labels[i]]):
            accuracy[4] += 1
    return accuracy / len(Y)

In [4]:
# caffe accuracy
threshold = 0.2
iter_Y = np.load('../test_caffe.npy')
np.array(map(get_accuracy, iter_Y), dtype=np.float32)

array([[ 0.35273159,  0.07897862,  0.07897862,  0.18349169,  0.18349169],
       [ 0.39964372,  0.1567696 ,  0.1567696 ,  0.20071259,  0.20071259],
       [ 0.41033253,  0.15558195,  0.15558195,  0.20130642,  0.20130642],
       [ 0.4287411 ,  0.16508314,  0.16508314,  0.20546319,  0.20546319],
       [ 0.43764845,  0.16745843,  0.16745843,  0.21199524,  0.21199524],
       [ 0.43527317,  0.16745843,  0.16745843,  0.21021378,  0.21021378],
       [ 0.43942994,  0.16745843,  0.16745843,  0.20961995,  0.20961995],
       [ 0.43824229,  0.17042756,  0.17042756,  0.21080761,  0.21080761],
       [ 0.43824229,  0.17042756,  0.17042756,  0.21140142,  0.21140142],
       [ 0.43824229,  0.17042756,  0.17042756,  0.21140142,  0.21140142]], dtype=float32)

In [5]:
# svm accuracy
threshold = 0
kernel_Y = np.load('../test_svm.npy')
np.array(map(get_accuracy, kernel_Y), dtype=np.float32)

array([[ 0.64073634,  0.33432305,  0.33432305,  0.65439433,  0.65439433],
       [ 0.72684085,  0.28800476,  0.28800476,  0.8497625 ,  0.8497625 ],
       [ 0.72921616,  0.35213777,  0.35213777,  0.86163896,  0.86163896]], dtype=float32)

In [6]:
def to_crf(X):
    def to_crf_step(x):
        scores = map(lambda s: x[s].sum(), state_space)
        return state_space[np.argmax(scores)]
    return np.array(map(to_crf_step, X), dtype=np.bool)

In [7]:
# crf accuracy
threshold = 0
kernel_Y_crf = np.array(map(to_crf, kernel_Y), dtype=np.bool)
np.array(map(get_accuracy, kernel_Y_crf), dtype=np.float32)

array([[ 0.41567695,  0.39904988,  0.39904988,  0.80047506,  0.80047506],
       [ 0.31057006,  0.28859857,  0.28859857,  0.89133018,  0.89133018],
       [ 0.37707838,  0.35510689,  0.35510689,  0.89251781,  0.89251781]], dtype=float32)